In [2]:
import os

In [3]:
pwd

'c:\\Deepankar\\Projects\\Text-Summarizer\\research'

In [4]:
os.chdir('../')

In [5]:
pwd

'c:\\Deepankar\\Projects\\Text-Summarizer'

In [6]:
# 3. Update entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [21]:
# 4. Update the configuration manager in src config

from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [22]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            params_filepath= PARAMS_FILE_PATH,            
            ) -> None:
        
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)->DataIngestionConfig:
        config= self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config= DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config

In [23]:
# 5. Update the components

import os
import urllib.request as request
import zipfile
from TextSummarizer.logging import logger
from TextSummarizer.utils.common import get_size

In [24]:
class DataIngestion:
    def __init__(self, data_ingestion_config: DataIngestionConfig) -> None:
        self.data_ingestion_config= data_ingestion_config


    def download_file(self):
        if not os.path.exists(self.data_ingestion_config.local_data_file):
            filename, headers= request.urlretrieve(
                url= self.data_ingestion_config.source_URL,
                filename= self.data_ingestion_config.local_data_file
            )
            logger.info(f'{filename} downloaded ! with following info: \n{headers}')
        
        else:
            logger.info(f'File already exists of size: {get_size(Path(self.data_ingestion_config.local_data_file))}')


    def extract_zip_file(self):
        '''
        zip_file_path: str
        Extracts the zip file into the data directory 
        Function returns None
        '''
        unzip_path= self.data_ingestion_config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.data_ingestion_config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f'Unzipped {self.data_ingestion_config.local_data_file} at {unzip_path} successfully.')
            # print(f'file unzipped at-path= {unzip_path}')

In [35]:
try:
    config= ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    print(data_ingestion_config.source_URL)
    data_ingestion= DataIngestion(data_ingestion_config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e :
    # e.__traceback__
    raise e

[2023-07-13 12:02:01,776: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-13 12:02:01,779: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-13 12:02:01,780: INFO: common: created directory at: artifacts]
[2023-07-13 12:02:01,782: INFO: common: created directory at: artifacts/data_ingestion]
https://github.com/ideepankarsharma2003/Text-Summarizer/raw/main/data/summarizer-data.zip
[2023-07-13 12:06:58,350: INFO: 3450869319: artifacts/data_ingestion/data.zip downloaded ! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4E02:773B:6E7FEA:816F97:64AF9A63
Accept-Ranges: bytes
Date

In [17]:
# try:
#     config= ConfigurationManager()
#     data_ingestion_config= config.get_data_ingestion_config()
#     data_ingestion= DataIngestion(data_ingestion_config= data_ingestion_config)
#     data_ingestion.download_file()
#     data_ingestion.extract_zip_file()
# except Exception as e :
#     print(e.__traceback__)

[2023-07-13 11:37:19,851: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-13 11:37:19,853: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-13 11:37:19,855: INFO: common: created directory at: artifacts]
[2023-07-13 11:37:19,856: INFO: common: created directory at: artifacts/data_ingestion]
[2023-07-13 11:37:19,858: INFO: 3450869319: File already exists of size: ~ 118 KB]
